In [1]:
from QCutils import *
from Database.QCDB import *
import numpy as np
import numpy as jnp

In [2]:
N = 6
beta = 1
J = 0.5
h = 0.3

In [3]:
#def createH(J,h)

H = jnp.zeros((2**N,2**N))

initLattice = jnp.kron(sigma_z,sigma_z)
for i in range(2,N):
    initLattice = jnp.kron(initLattice,jnp.eye(2))

H += - J * initLattice

for lattice_point in range(1,N-1):
    curr = jnp.eye(2)
    for i in range(1,lattice_point):
        curr = jnp.kron(curr,jnp.eye(2))
    curr = jnp.kron( jnp.kron(curr,sigma_z),sigma_z)
    for i in range(lattice_point+2,N):
        curr = jnp.kron(curr,jnp.eye(2))
    
    assert curr.shape[0] == H.shape[0]
    
    H += -J * curr


initLattice = sigma_x
for i in range(1,N):
    initLattice = jnp.kron(initLattice,jnp.eye(2))

H += - h * initLattice

for lattice_point in range(1,N-1):
    curr = jnp.eye(2)
    for i in range(1,lattice_point):
        curr = jnp.kron(curr,jnp.eye(2))
    curr = jnp.kron(curr,sigma_x)
    for i in range(lattice_point+1,N):
        curr = jnp.kron(curr,jnp.eye(2))
    
    assert curr.shape[0] == H.shape[0]
    
    H += -h * curr

H = jnp.array(H,dtype=jnp.complex128)

In [4]:
test1 = oneMoreCirc(N)
test1.setHamiltonian(H)
test1.setBeta(beta)

In [5]:
for layer in range(4):
    for i in range(N-1):
        test1.add_Arbitrary_gate((i,i+1))
    for j in range(N-1):
        i = (N-2 - j)
        test1.add_Arbitrary_gate((i,i+1))

In [6]:
test1.construct()

In [7]:
def ValandGrad(thetas):
    test1.updateThetas(thetas)
    return test1.calc(), test1.getThetaGrad()

In [8]:
import scipy.optimize
initThetas = 2* np.pi * np.random.rand(test1.size())
results = scipy.optimize.minimize(ValandGrad, initThetas,
                                  method='L-BFGS-B', jac=True)
print("success:", results.success, "\nniterations:", results.nit, "\nfinal loss:", results.fun)

/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/qwe/MLPhys/QC/pure/QCutils.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
  thetaGrad[i] = grad1
/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,


KeyboardInterrupt: 

In [ ]:
-4.509538109296834+0j

In [20]:
saveTojson("QC with one more qubit",beta=beta,additional={"layer":3},res=float(results.fun))

/var/folders/2_/8rs1gbh95pd73jb7n34phc980000gn/T/ipykernel_7599/1750703168.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  saveTojson("QC with one more qubit",beta=beta,additional={"layer":3},res=float(results.fun))


TypeError: the JSON object must be str, bytes or bytearray, not TextIOWrapper

In [13]:
from scipy.linalg import expm
-np.log(np.trace(expm(- beta * H)))

(-4.9530060181077005-0j)

In [21]:
learningRate = 0.05
maxStep = 100
record = np.zeros(maxStep)

thetas = 2* np.pi * np.random.rand(test1.size())

for i in range(maxStep):
    test1.updateThetas(thetas)
    record[i] = test1.calc()
    dTheta = test1.getThetaGrad()
    thetas -= learningRate * dTheta

/tmp/ipykernel_10766/1987415637.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  record[i] = test1.calc()


In [16]:
record[record != 0][0]

-1.9857282192311847

In [9]:
r = 0.5
circList = test1.genGradCircList()
thetaGrad = np.zeros(test1.size())

In [10]:
circPlus, circMinus = circList[1]

In [15]:
grad1 = r * (circPlus.calc() - circMinus.calc())
grad1[0][0]

(-0.276411996093495+0j)

In [11]:
circPlus, circMinus = circList[1]
grad1 = r * (circPlus.calc() - circMinus.calc())
thetaGrad[1] = grad1


In [9]:
lst = test1.genGradCircList()

In [12]:
len(lst)
circPlus, circMinus = lst[0]

In [16]:
circMinus.calc()
grad = 0.5 * (circPlus.calc() - circMinus.calc())
grad

/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


array([[-0.01312868+0.j]])

In [ ]:
test1.getThetaGrad()